In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
import numpy as np

Load Dataset:<br>
train_dataset=original training set with no augmentation<br>
back_dataset=training set with back-translation<br>
aug_dataset=training set with eda-paraphrasing augmentation

In [2]:
train_dataset=pd.read_csv(r'..\DAIC\Preprocessed\train_dataset.csv')
back_dataset=pd.read_csv(r'..\DAIC\Preprocessed\back_dataset.csv')
aug_dataset=pd.read_csv(r'..\DAIC\Preprocessed\aug_dataset.csv')
test_dataset=pd.read_csv(r'..\DAIC\Preprocessed\test_dataset.csv')
val_dataset=pd.read_csv(r'..\DAIC\Preprocessed\dev_dataset.csv')

Checking the no of datapoints and class balances.

In [3]:
print(f'The samples in training dataset is: ',(len(train_dataset['response'])),'and the distribution is ',(train_dataset['PHQ8_Binary'].value_counts()))
print(f'The samples in back dataset is: ',(len(back_dataset['response'])),'and the distribution is ',(back_dataset['PHQ8_Binary'].value_counts()))
print(f'The samples in aug dataset is: ',(len(aug_dataset['response'])),'and the distribution is ',(aug_dataset['PHQ8_Binary'].value_counts()))
print(f'The samples in validation dataset is: ',(len(val_dataset['response'])),'and the distribution is ',(val_dataset['PHQ8_Binary'].value_counts()))
print(f'The samples in test dataset is: ',(len(test_dataset['response'])),'and the distribution is ',(test_dataset['PHQ8_Binary'].value_counts()))

The samples in training dataset is:  107 and the distribution is  PHQ8_Binary
0    77
1    30
Name: count, dtype: int64
The samples in back dataset is:  136 and the distribution is  PHQ8_Binary
0    77
1    59
Name: count, dtype: int64
The samples in aug dataset is:  127 and the distribution is  PHQ8_Binary
0    77
1    50
Name: count, dtype: int64
The samples in validation dataset is:  35 and the distribution is  PHQ8_Binary
0    23
1    12
Name: count, dtype: int64
The samples in test dataset is:  47 and the distribution is  PHQ8_Binary
0    33
1    14
Name: count, dtype: int64


Let's start with the training dataset. First of all we will do the classification without under/oversampling, using tf-idf, word2vec and glove. After which we will use sampling balancing. We will the test in the validation set and subsequently test set. We will do the same for back and aug dataset. At last we, will also try incorporating val into training dataset, as we have a separate test dataset for testing.

<h1>TF-IDF</h1>
<h3>Train_set</h3>

In [4]:
tfidf_vectorizer=TfidfVectorizer(lowercase=True,stop_words='english',max_features=6100)

X_train_tfidf=tfidf_vectorizer.fit_transform(train_dataset['response'])
X_val_tfidf=tfidf_vectorizer.transform(val_dataset['response'])
X_test_tfidf=tfidf_vectorizer.transform(test_dataset['response'])

y_train=train_dataset['PHQ8_Binary']
y_val=val_dataset['PHQ8_Binary']
y_test=test_dataset['PHQ8_Binary']

In [5]:
ratio = y_train.value_counts()[0] / y_train.value_counts()[1]
xg = xgb.XGBClassifier(n_estimators=7,reg_lambda=1.2,use_label_encoder=False, eval_metric='error', random_state=4, scale_pos_weight=ratio)
xg.fit(X_train_tfidf,y_train)
y_val_pred=xg.predict(X_val_tfidf)
y_test_pred=xg.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.72      0.57      0.63        23
  Depression       0.41      0.58      0.48        12

    accuracy                           0.57        35
   macro avg       0.57      0.57      0.56        35
weighted avg       0.62      0.57      0.58        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.64      0.64      0.64        33
  Depression       0.14      0.14      0.14        14

    accuracy                           0.49        47
   macro avg       0.39      0.39      0.39        47
weighted avg       0.49      0.49      0.49        47



In [6]:
#Random Undersampling
rus=RandomUnderSampler(random_state=1)
X_train_tfidf_un,y_train_un=rus.fit_resample(X_train_tfidf,y_train)

xg = xgb.XGBClassifier(n_estimators=23,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=42)
xg.fit(X_train_tfidf_un,y_train_un)

y_val_pred=xg.predict(X_val_tfidf)
y_test_pred=xg.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.71      0.52      0.60        23
  Depression       0.39      0.58      0.47        12

    accuracy                           0.54        35
   macro avg       0.55      0.55      0.53        35
weighted avg       0.60      0.54      0.55        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.77      0.52      0.62        33
  Depression       0.36      0.64      0.46        14

    accuracy                           0.55        47
   macro avg       0.57      0.58      0.54        47
weighted avg       0.65      0.55      0.57        47



In [7]:
#Random OverSampling
smote=SMOTE(random_state=9)
X_train_tfidf_smote,y_train_smote=smote.fit_resample(X_train_tfidf,y_train)
xg = xgb.XGBClassifier(n_estimators=23,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=42)
xg.fit(X_train_tfidf_smote,y_train_smote)

y_val_pred=xg.predict(X_val_tfidf)
y_test_pred=xg.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.73      0.83      0.78        23
  Depression       0.56      0.42      0.48        12

    accuracy                           0.69        35
   macro avg       0.64      0.62      0.63        35
weighted avg       0.67      0.69      0.67        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.74      0.79      0.76        33
  Depression       0.42      0.36      0.38        14

    accuracy                           0.66        47
   macro avg       0.58      0.57      0.57        47
weighted avg       0.65      0.66      0.65        47



<h3>Back_Set</h3>

In [8]:
tfidf_vectorizer=TfidfVectorizer(lowercase=True,stop_words='english',max_features=6100)

X_back_tfidf=tfidf_vectorizer.fit_transform(back_dataset['response'])
X_val_tfidf=tfidf_vectorizer.transform(val_dataset['response'])
X_test_tfidf=tfidf_vectorizer.transform(test_dataset['response'])

y_back=back_dataset['PHQ8_Binary']
y_val=val_dataset['PHQ8_Binary']
y_test=test_dataset['PHQ8_Binary']

In [9]:
ratio = y_back.value_counts()[0] / y_back.value_counts()[1]
xg = xgb.XGBClassifier(n_estimators=13,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3,scale_pos_weight=ratio)
xg.fit(X_back_tfidf,y_back)

y_val_pred=xg.predict(X_val_tfidf)
y_test_pred=xg.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.67      0.61      0.64        23
  Depression       0.36      0.42      0.38        12

    accuracy                           0.54        35
   macro avg       0.51      0.51      0.51        35
weighted avg       0.56      0.54      0.55        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.70      0.70      0.70        33
  Depression       0.29      0.29      0.29        14

    accuracy                           0.57        47
   macro avg       0.49      0.49      0.49        47
weighted avg       0.57      0.57      0.57        47



In [20]:
#Random Undersampling
rus=RandomUnderSampler(random_state=5)
X_back_tfidf_un,y_back_un=rus.fit_resample(X_back_tfidf,y_back)
xg = xgb.XGBClassifier(n_estimators=35,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=32)

xg.fit(X_back_tfidf_un,y_back_un)

y_val_pred=xg.predict(X_val_tfidf)
y_test_pred=xg.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.65      0.65      0.65        23
  Depression       0.33      0.33      0.33        12

    accuracy                           0.54        35
   macro avg       0.49      0.49      0.49        35
weighted avg       0.54      0.54      0.54        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.71      0.76      0.74        33
  Depression       0.33      0.29      0.31        14

    accuracy                           0.62        47
   macro avg       0.52      0.52      0.52        47
weighted avg       0.60      0.62      0.61        47



In [28]:
#Random OverSampling
smote=SMOTE()
X_back_tfidf_smote,y_back_smote=smote.fit_resample(X_back_tfidf,y_back)
xg = xgb.XGBClassifier(n_estimators=125,reg_lambda=0.02,use_label_encoder=False, eval_metric='error', random_state=32)
xg.fit(X_back_tfidf_smote,y_back_smote)

y_val_pred=xg.predict(X_val_tfidf)
y_test_pred=xg.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.65      0.67        23
  Depression       0.38      0.42      0.40        12

    accuracy                           0.57        35
   macro avg       0.53      0.53      0.53        35
weighted avg       0.58      0.57      0.58        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.79      0.73        33
  Depression       0.22      0.14      0.17        14

    accuracy                           0.60        47
   macro avg       0.45      0.47      0.45        47
weighted avg       0.55      0.60      0.57        47



<h3>Aug_Set</h3>

In [29]:
tfidf_vectorizer=TfidfVectorizer(lowercase=True,stop_words='english',max_features=6200)

X_aug_tfidf=tfidf_vectorizer.fit_transform(aug_dataset['response'])
X_val_tfidf=tfidf_vectorizer.transform(val_dataset['response'])
X_test_tfidf=tfidf_vectorizer.transform(test_dataset['response'])

y_aug=aug_dataset['PHQ8_Binary']
y_val=val_dataset['PHQ8_Binary']
y_test=test_dataset['PHQ8_Binary']

In [38]:
ratio = y_aug.value_counts()[0] / y_aug.value_counts()[1]
xg = xgb.XGBClassifier(n_estimators=13,reg_lambda=1.0,use_label_encoder=False, eval_metric='error', random_state=1,scale_pos_weight=ratio)
xg.fit(X_aug_tfidf,y_aug)

y_val_pred=xg.predict(X_val_tfidf)
y_test_pred=xg.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.67      0.70      0.68        23
  Depression       0.36      0.33      0.35        12

    accuracy                           0.57        35
   macro avg       0.52      0.51      0.51        35
weighted avg       0.56      0.57      0.57        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.70      0.69        33
  Depression       0.23      0.21      0.22        14

    accuracy                           0.55        47
   macro avg       0.45      0.46      0.45        47
weighted avg       0.54      0.55      0.55        47



In [42]:
#Random Undersampling
rus=RandomUnderSampler(random_state=5)
X_aug_tfidf_un,y_aug_un=rus.fit_resample(X_aug_tfidf,y_aug)
xg = xgb.XGBClassifier(n_estimators=1333,reg_lambda=200.0,use_label_encoder=False, eval_metric='error', random_state=3)
# rf=LogisticRegression(max_iter=1000)
xg.fit(X_aug_tfidf_un,y_aug_un)

y_val_pred=xg.predict(X_val_tfidf)
y_test_pred=xg.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.62      0.57      0.59        23
  Depression       0.29      0.33      0.31        12

    accuracy                           0.49        35
   macro avg       0.45      0.45      0.45        35
weighted avg       0.50      0.49      0.49        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.71      0.76      0.74        33
  Depression       0.33      0.29      0.31        14

    accuracy                           0.62        47
   macro avg       0.52      0.52      0.52        47
weighted avg       0.60      0.62      0.61        47



In [45]:
#Random OverSampling
smote=SMOTE(random_state=5)
X_aug_tfidf_smote,y_aug_smote=smote.fit_resample(X_aug_tfidf,y_aug)
xg = xgb.XGBClassifier(n_estimators=13,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3)
xg.fit(X_aug_tfidf_smote,y_aug_smote)

y_val_pred=xg.predict(X_val_tfidf)
y_test_pred=xg.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.74      0.71        23
  Depression       0.40      0.33      0.36        12

    accuracy                           0.60        35
   macro avg       0.54      0.54      0.54        35
weighted avg       0.58      0.60      0.59        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.69      0.61      0.65        33
  Depression       0.28      0.36      0.31        14

    accuracy                           0.53        47
   macro avg       0.48      0.48      0.48        47
weighted avg       0.57      0.53      0.55        47



<h1>Word2Vec</h1>
<h3>Train_set</h3>

In [46]:
import gensim

word2vec_path='..\GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin'
word2vec=gensim.models.KeyedVectors.load_word2vec_format(word2vec_path,binary=True)

In [47]:
def get_average_word2vec(tokens_list,vector,k=300):
    valid_vectors=[vector[word] for word in tokens_list if word in vector]

    if not valid_vectors:
        print(tokens_list)
        return np.zeros(k)
        
    
    return np.mean(valid_vectors,axis=0)

In [48]:
train_dataset['tokens']=train_dataset['response'].apply(lambda x:x.split())
val_dataset['tokens']=val_dataset['response'].apply(lambda x:x.split())
test_dataset['tokens']=test_dataset['response'].apply(lambda x:x.split())
X_train_word2vec=np.array([get_average_word2vec(tokens,word2vec) for tokens in train_dataset['tokens'] ])
X_val_word2vec=np.array([get_average_word2vec(tokens,word2vec) for tokens in val_dataset['tokens']])
X_test_word2vec=np.array([get_average_word2vec(tokens,word2vec) for tokens in test_dataset['tokens']])

In [52]:
ratio = y_train.value_counts()[0] / y_train.value_counts()[1]
xg = xgb.XGBClassifier(n_estimators=31,reg_lambda=1.0,use_label_encoder=False, eval_metric='error', random_state=3,scale_pos_weight=ratio)
xg.fit(X_train_word2vec,y_train)

y_val_pred=xg.predict(X_val_word2vec)
y_test_pred=xg.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.66      0.91      0.76        23
  Depression       0.33      0.08      0.13        12

    accuracy                           0.63        35
   macro avg       0.49      0.50      0.45        35
weighted avg       0.55      0.63      0.55        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.74      0.88      0.81        33
  Depression       0.50      0.29      0.36        14

    accuracy                           0.70        47
   macro avg       0.62      0.58      0.58        47
weighted avg       0.67      0.70      0.67        47



In [53]:
#Random Undersampling
rus=RandomUnderSampler(random_state=42)
X_train_word2vec_un,y_train_word2vec_un=rus.fit_resample(X_train_word2vec,y_train)

In [64]:
rus=RandomUnderSampler(random_state=5)
X_train_word2vec_un,y_train_word2vec_un=rus.fit_resample(X_train_word2vec,y_train)

xg = xgb.XGBClassifier(n_estimators=31,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3)
# rf=LogisticRegression(max_iter=1000)
xg.fit(X_train_word2vec_un,y_train_word2vec_un)

y_val_pred=xg.predict(X_val_word2vec)
y_test_pred=xg.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.72      0.57      0.63        23
  Depression       0.41      0.58      0.48        12

    accuracy                           0.57        35
   macro avg       0.57      0.57      0.56        35
weighted avg       0.62      0.57      0.58        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.75      0.64      0.69        33
  Depression       0.37      0.50      0.42        14

    accuracy                           0.60        47
   macro avg       0.56      0.57      0.56        47
weighted avg       0.64      0.60      0.61        47



In [69]:
#Random OverSampling
smote=SMOTE(random_state=5)
X_train_word2vec_smote,y_train_word2vec_smote=smote.fit_resample(X_train_word2vec,y_train)
xg = xgb.XGBClassifier(n_estimators=13,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3)
xg.fit(X_train_word2vec_smote,y_train_word2vec_smote)

y_val_pred=xg.predict(X_val_word2vec)
y_test_pred=xg.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.70      0.91      0.79        23
  Depression       0.60      0.25      0.35        12

    accuracy                           0.69        35
   macro avg       0.65      0.58      0.57        35
weighted avg       0.67      0.69      0.64        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.73      0.82      0.77        33
  Depression       0.40      0.29      0.33        14

    accuracy                           0.66        47
   macro avg       0.56      0.55      0.55        47
weighted avg       0.63      0.66      0.64        47



<h3>Back_set</h3>

In [70]:
back_dataset['tokens']=back_dataset['response'].apply(lambda x:x.split())
X_back_word2vec=np.array([get_average_word2vec(tokens,word2vec) for tokens in back_dataset['tokens'] ])

In [74]:
ratio = y_back.value_counts()[0] / y_back.value_counts()[1]
xg = xgb.XGBClassifier(n_estimators=33,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3,scale_pos_weight=ratio)
xg.fit(X_back_word2vec,y_back)

y_val_pred=xg.predict(X_val_word2vec)
y_test_pred=xg.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.67      0.87      0.75        23
  Depression       0.40      0.17      0.24        12

    accuracy                           0.63        35
   macro avg       0.53      0.52      0.50        35
weighted avg       0.58      0.63      0.58        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.69      0.88      0.77        33
  Depression       0.20      0.07      0.11        14

    accuracy                           0.64        47
   macro avg       0.45      0.48      0.44        47
weighted avg       0.54      0.64      0.57        47



In [75]:
#Random Undersampling
rus=RandomUnderSampler(random_state=5)
X_back_word2vec_un,y_back_word2vec_un=rus.fit_resample(X_back_word2vec,y_back)

In [81]:
xg = xgb.XGBClassifier(n_estimators=3,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3)
xg.fit(X_back_word2vec_un,y_back_word2vec_un)

y_val_pred=xg.predict(X_val_word2vec)
y_test_pred=xg.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.62      0.65      0.64        23
  Depression       0.27      0.25      0.26        12

    accuracy                           0.51        35
   macro avg       0.45      0.45      0.45        35
weighted avg       0.50      0.51      0.51        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.71      0.73      0.72        33
  Depression       0.31      0.29      0.30        14

    accuracy                           0.60        47
   macro avg       0.51      0.51      0.51        47
weighted avg       0.59      0.60      0.59        47



In [83]:
#Random OverSampling
smote=SMOTE(random_state=7)
X_back_word2vec_smote,y_back_word2vec_smote=smote.fit_resample(X_back_word2vec,y_back)
xg = xgb.XGBClassifier(n_estimators=3,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3)
xg.fit(X_back_word2vec_smote,y_back_word2vec_smote)

y_val_pred=xg.predict(X_val_word2vec)
y_test_pred=xg.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.70      0.70      0.70        23
  Depression       0.42      0.42      0.42        12

    accuracy                           0.60        35
   macro avg       0.56      0.56      0.56        35
weighted avg       0.60      0.60      0.60        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.70      0.69        33
  Depression       0.23      0.21      0.22        14

    accuracy                           0.55        47
   macro avg       0.45      0.46      0.45        47
weighted avg       0.54      0.55      0.55        47



<h3>Aug_set</h3>

In [84]:
aug_dataset['tokens']=aug_dataset['response'].apply(lambda x:x.split())
X_aug_word2vec=np.array([get_average_word2vec(tokens,word2vec) for tokens in aug_dataset['tokens'] ])

In [87]:
ratio = y_aug.value_counts()[0] / y_aug.value_counts()[1]
xg = xgb.XGBClassifier(n_estimators=33,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3,scale_pos_weight=ratio)
xg.fit(X_aug_word2vec,y_aug)

y_val_pred=xg.predict(X_val_word2vec)
y_test_pred=xg.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.73      0.83      0.78        23
  Depression       0.56      0.42      0.48        12

    accuracy                           0.69        35
   macro avg       0.64      0.62      0.63        35
weighted avg       0.67      0.69      0.67        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.79      0.73        33
  Depression       0.22      0.14      0.17        14

    accuracy                           0.60        47
   macro avg       0.45      0.47      0.45        47
weighted avg       0.55      0.60      0.57        47



In [88]:
#Random Undersampling
rus=RandomUnderSampler(random_state=5)
X_aug_word2vec_un,y_aug_word2vec_un=rus.fit_resample(X_aug_word2vec,y_aug)

In [90]:
xg = xgb.XGBClassifier(n_estimators=3,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3)
xg.fit(X_aug_word2vec_un,y_aug_word2vec_un)

y_val_pred=xg.predict(X_val_word2vec)
y_test_pred=xg.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.65      0.67        23
  Depression       0.38      0.42      0.40        12

    accuracy                           0.57        35
   macro avg       0.53      0.53      0.53        35
weighted avg       0.58      0.57      0.58        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.77      0.82      0.79        33
  Depression       0.50      0.43      0.46        14

    accuracy                           0.70        47
   macro avg       0.64      0.62      0.63        47
weighted avg       0.69      0.70      0.70        47



In [92]:
#Random OverSampling
smote=SMOTE(random_state=7)
X_aug_word2vec_smote,y_aug_word2vec_smote=smote.fit_resample(X_aug_word2vec,y_aug)
xg = xgb.XGBClassifier(n_estimators=3,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3)
xg.fit(X_aug_word2vec_smote,y_aug_word2vec_smote)

y_val_pred=xg.predict(X_val_word2vec)
y_test_pred=xg.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.70      0.83      0.76        23
  Depression       0.50      0.33      0.40        12

    accuracy                           0.66        35
   macro avg       0.60      0.58      0.58        35
weighted avg       0.63      0.66      0.64        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.82      0.74        33
  Depression       0.14      0.07      0.10        14

    accuracy                           0.60        47
   macro avg       0.41      0.44      0.42        47
weighted avg       0.52      0.60      0.55        47



<h1>GlOvE</h1>
<h3>Train_Set</h3>

In [93]:
embedding_index={}
glove_path='../glove.6B.100d.txt'
with open(glove_path,'r',encoding='utf-8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        coefs=np.asarray(values[1:],dtype='float32')
        embedding_index[word]=coefs

In [94]:
def get_average_glove(tokens_list,embedding_index,k=100):
    valid_vectors=[embedding_index[word] for word in tokens_list if word in embedding_index]

    if not valid_vectors:
        return np.zeros(k)
    
    return np.mean(valid_vectors,axis=0)

In [95]:
X_train_glove=np.array([get_average_glove(tokens,embedding_index) for tokens in train_dataset['tokens']])
X_back_glove=np.array([get_average_glove(tokens,embedding_index) for tokens in back_dataset['tokens']])
X_aug_glove=np.array([get_average_glove(tokens,embedding_index) for tokens in aug_dataset['tokens']])
X_val_glove=np.array([get_average_glove(tokens,embedding_index) for tokens in val_dataset['tokens']])
X_test_glove=np.array([get_average_glove(tokens,embedding_index) for tokens in test_dataset['tokens']])

In [101]:
ratio = y_train.value_counts()[0] / y_train.value_counts()[1]
xg = xgb.XGBClassifier(n_estimators=3,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3,scale_pos_weight=ratio)
xg.fit(X_train_glove,y_train)

y_val_pred=xg.predict(X_val_glove)
y_test_pred=xg.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.65      0.67        23
  Depression       0.38      0.42      0.40        12

    accuracy                           0.57        35
   macro avg       0.53      0.53      0.53        35
weighted avg       0.58      0.57      0.58        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.67      0.73      0.70        33
  Depression       0.18      0.14      0.16        14

    accuracy                           0.55        47
   macro avg       0.42      0.44      0.43        47
weighted avg       0.52      0.55      0.54        47



In [102]:
#Random Undersampling
rus=RandomUnderSampler(random_state=5)
X_train_glove_un,y_train_glove_un=rus.fit_resample(X_train_glove,y_train)

In [104]:
xg = xgb.XGBClassifier(n_estimators=3,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3)
xg.fit(X_train_glove_un,y_train_glove_un)

y_val_pred=xg.predict(X_val_glove)
y_test_pred=xg.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.86      0.52      0.65        23
  Depression       0.48      0.83      0.61        12

    accuracy                           0.63        35
   macro avg       0.67      0.68      0.63        35
weighted avg       0.73      0.63      0.63        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.52      0.33      0.41        33
  Depression       0.15      0.29      0.20        14

    accuracy                           0.32        47
   macro avg       0.34      0.31      0.30        47
weighted avg       0.41      0.32      0.35        47



In [113]:
#Random OverSampling
smote=SMOTE(random_state=9)
X_train_glove_smote,y_train_glove_smote=smote.fit_resample(X_train_glove,y_train)
xg = xgb.XGBClassifier(n_estimators=5,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3)
xg.fit(X_train_glove_smote,y_train_glove_smote)

y_val_pred=xg.predict(X_val_glove)
y_test_pred=xg.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.62      0.70      0.65        23
  Depression       0.22      0.17      0.19        12

    accuracy                           0.51        35
   macro avg       0.42      0.43      0.42        35
weighted avg       0.48      0.51      0.49        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.74      0.88      0.81        33
  Depression       0.50      0.29      0.36        14

    accuracy                           0.70        47
   macro avg       0.62      0.58      0.58        47
weighted avg       0.67      0.70      0.67        47



<h3>Back_set</h3>

In [127]:
ratio = y_back.value_counts()[0] / y_back.value_counts()[1]
xg = xgb.XGBClassifier(n_estimators=3,reg_lambda=20.0,use_label_encoder=False, eval_metric='error', random_state=3,scale_pos_weight=ratio)
xg.fit(X_back_glove,y_back)

y_val_pred=xg.predict(X_val_glove)
y_test_pred=xg.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.65      0.65      0.65        23
  Depression       0.33      0.33      0.33        12

    accuracy                           0.54        35
   macro avg       0.49      0.49      0.49        35
weighted avg       0.54      0.54      0.54        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.69      0.76      0.72        33
  Depression       0.27      0.21      0.24        14

    accuracy                           0.60        47
   macro avg       0.48      0.49      0.48        47
weighted avg       0.57      0.60      0.58        47



In [128]:
#Random Undersampling
rus=RandomUnderSampler(random_state=5)
X_back_glove_un,y_back_glove_un=rus.fit_resample(X_back_glove,y_back)

In [135]:
xg = xgb.XGBClassifier(n_estimators=17,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3)
# rf=LogisticRegression(max_iter=1000)
xg.fit(X_back_glove_un,y_back_glove_un)

y_val_pred=xg.predict(X_val_glove)
y_test_pred=xg.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.60      0.65      0.63        23
  Depression       0.20      0.17      0.18        12

    accuracy                           0.49        35
   macro avg       0.40      0.41      0.40        35
weighted avg       0.46      0.49      0.47        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.70      0.79      0.74        33
  Depression       0.30      0.21      0.25        14

    accuracy                           0.62        47
   macro avg       0.50      0.50      0.50        47
weighted avg       0.58      0.62      0.60        47



In [139]:
#Random OverSampling
smote=SMOTE(random_state=1)
X_back_glove_smote,y_back_glove_smote=smote.fit_resample(X_back_glove,y_back)
xg = xgb.XGBClassifier(n_estimators=1,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3)
xg.fit(X_back_glove_smote,y_back_glove_smote)

y_val_pred=xg.predict(X_val_glove)
y_test_pred=xg.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.69      0.78      0.73        23
  Depression       0.44      0.33      0.38        12

    accuracy                           0.63        35
   macro avg       0.57      0.56      0.56        35
weighted avg       0.61      0.63      0.61        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.76      0.85      0.80        33
  Depression       0.50      0.36      0.42        14

    accuracy                           0.70        47
   macro avg       0.63      0.60      0.61        47
weighted avg       0.68      0.70      0.69        47



<h3>Aug_set</h3>

In [143]:
ratio = y_aug.value_counts()[0] / y_aug.value_counts()[1]
xg = xgb.XGBClassifier(n_estimators=5,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3,scale_pos_weight=ratio)
xg.fit(X_aug_glove,y_aug)

y_val_pred=xg.predict(X_val_glove)
y_test_pred=xg.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.74      0.71        23
  Depression       0.40      0.33      0.36        12

    accuracy                           0.60        35
   macro avg       0.54      0.54      0.54        35
weighted avg       0.58      0.60      0.59        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.79      0.73        33
  Depression       0.22      0.14      0.17        14

    accuracy                           0.60        47
   macro avg       0.45      0.47      0.45        47
weighted avg       0.55      0.60      0.57        47



In [144]:
#Random Undersampling
rus=RandomUnderSampler(random_state=5)
X_aug_glove_un,y_aug_glove_un=rus.fit_resample(X_aug_glove,y_aug)

In [146]:
xg = xgb.XGBClassifier(n_estimators=1,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3)
xg.fit(X_aug_glove_un,y_aug_glove_un)

y_val_pred=xg.predict(X_val_glove)
y_test_pred=xg.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.71      0.65      0.68        23
  Depression       0.43      0.50      0.46        12

    accuracy                           0.60        35
   macro avg       0.57      0.58      0.57        35
weighted avg       0.62      0.60      0.61        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.72      0.79      0.75        33
  Depression       0.36      0.29      0.32        14

    accuracy                           0.64        47
   macro avg       0.54      0.54      0.54        47
weighted avg       0.62      0.64      0.62        47



In [148]:
#Random OverSampling
smote=SMOTE(random_state=7)
X_aug_glove_smote,y_aug_glove_smote=smote.fit_resample(X_aug_glove,y_aug)
xg = xgb.XGBClassifier(n_estimators=1,reg_lambda=2.0,use_label_encoder=False, eval_metric='error', random_state=3)
xg.fit(X_aug_glove_smote,y_aug_glove_smote)

y_val_pred=xg.predict(X_val_glove)
y_test_pred=xg.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.71      0.65      0.68        23
  Depression       0.43      0.50      0.46        12

    accuracy                           0.60        35
   macro avg       0.57      0.58      0.57        35
weighted avg       0.62      0.60      0.61        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.73      0.82      0.77        33
  Depression       0.40      0.29      0.33        14

    accuracy                           0.66        47
   macro avg       0.56      0.55      0.55        47
weighted avg       0.63      0.66      0.64        47

